In [1]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,103 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [2]:
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver
!pip install chromedriver-autoinstaller

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [3]:
!python --version

import selenium
print(selenium.__version__)

Python 3.10.12
4.26.1


In [4]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from bs4 import BeautifulSoup
from urllib.request import urlretrieve

import time
import pandas as pd
import chromedriver_autoinstaller  # setup chrome options

In [5]:
chrome_path = "/usr/lib/chromium-browser/chromedriver"

In [6]:
sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chromedriver_autoinstaller.install()  # set the target URL

'/usr/local/lib/python3.10/dist-packages/chromedriver_autoinstaller/130/chromedriver'

In [7]:
driver = webdriver.Chrome(options=chrome_options)

In [8]:
from datetime import datetime, timedelta

# 오늘 날짜 계산
today = datetime.now().date()
ninety_days_ago = today - timedelta(days=90)

In [9]:
import re

In [19]:

url = "https://snapshot.bok.or.kr/"
driver.get(url)

time.sleep(3)

path_element = driver.find_element(By.CSS_SELECTOR, "a[href='/dashboard/E']")

path_element.text


'국제 금융시장'

In [35]:
# request_chart_type=['회사채 신용 스프레드']

url = "https://snapshot.bok.or.kr/dashboard/E1"
driver.get(url)

time.sleep(2)
target_element=driver.find_element(By.CSS_SELECTOR, 'div#chart967.chart_target')
time.sleep(2)
iframe=driver.find_element(By.CSS_SELECTOR, "iframe[src='/chart.html?id=967']")
driver.switch_to.frame(iframe)
path_element = driver.find_element(By.CSS_SELECTOR, "path.highcharts-graph")

d_value = path_element.get_attribute("d")


TypeError: expected string or bytes-like object

In [49]:
pattern = r"([ML])\s+([-]?\d+\.\d+)\s+([-]?\d+\.\d+)"
points = re.findall(pattern, d_value)

points

df = pd.DataFrame(points, columns=['ML','X', 'Y'])
df['X'] = df['X'].astype(float)
df['Y'] = df['Y'].astype(float)

display(df)

,ML,X,Y
0,M,-0.504608,143.48
1,L,0.504608,145.18
2,L,1.009217,143.14
3,L,1.513825,140.76
4,L,2.018433,142.46
...,...,...,...
1233,L,654.476959,60.86
1234,L,654.981567,59.50
1235,L,655.486175,58.48
1236,L,655.990783,58.48


In [53]:
driver.execute_script("""
    window.svgToScreenCoords = function(svg, x, y) {
        var pt = svg.createSVGPoint();
        pt.x = x;
        pt.y = y;
        var transformed = pt.matrixTransform(svg.getScreenCTM());
        return [transformed.x, transformed.y];
    }
""")

In [58]:
for x, y in list(zip(df['X'], df['Y'])):
    screen_coords = driver.execute_script("return window.svgToScreenCoords(arguments[0], arguments[1], arguments[2]);",
                                          d_value, x, y)
    action.move_to_element_with_offset(d_value, screen_coords[0], screen_coords[1]).perform()
    time.sleep(0.5)  # 툴팁이 나타날 때까지 대기

    tooltip = driver.find_element(By.CSS_SELECTOR, "g.highcharts-label.highcharts-tooltip.highcharts-color-0")

    tooltip_text = tooltip.text

    # 날짜 정보 추출 (정규표현식 사용)
    date_pattern = r'<tspan style="font-size: 10px;">(.*?)</tspan>'
    date_match = re.search(date_pattern, tooltip_text)
    date = date_match.group(1) if date_match else 'No Date Found'

    # 채권 정보 추출 (정규표현식 사용)
    bond_pattern = r'([가-힣 ]+\(\d+년\)): <tspan style="font-weight: bold;">(\d+\.\d+)</tspan>'
    bond_matches = re.findall(bond_pattern, tooltip_text)

    data.append({'Date': date, 'Value': bond_matches})

    print(f"Point: ({point['x']}, {point['y']}), Tooltip: {tooltip_text}")

# 날짜순으로 정렬
data.sort(key=lambda x: x['Date'])

# 4. 메인 프레임으로 복귀
driver.switch_to.default_content()

driver.quit()


JavascriptException: Message: javascript error: svg.createSVGPoint is not a function
  (Session info: chrome=130.0.6723.91)
Stacktrace:
#0 0x5aa309b7e13a <unknown>
#1 0x5aa309699460 <unknown>
#2 0x5aa30969f193 <unknown>
#3 0x5aa3096a10e4 <unknown>
#4 0x5aa309729a7c <unknown>
#5 0x5aa30970a012 <unknown>
#6 0x5aa309728af6 <unknown>
#7 0x5aa309709db3 <unknown>
#8 0x5aa3096d8c10 <unknown>
#9 0x5aa3096d9bee <unknown>
#10 0x5aa309b4a3cb <unknown>
#11 0x5aa309b4e368 <unknown>
#12 0x5aa309b37eec <unknown>
#13 0x5aa309b4eee7 <unknown>
#14 0x5aa309b1d13f <unknown>
#15 0x5aa309b6c858 <unknown>
#16 0x5aa309b6ca20 <unknown>
#17 0x5aa309b7cfb6 <unknown>
#18 0x7ac0595caac3 <unknown>


In [45]:
svg_element=driver.find_element(By.CSS_SELECTOR, "svg.highcharts-root")

for point in d_value[:90]:

  action = webdriver.ActionChains(driver)

  action.move_to_element_with_offset(path_element, point['X'], point['Y']).perform()

  # 잠시 대기하여 툴팁이 나타나게 함
  time.sleep(0.5)

  # 툴팁 내용 추출
  tooltip = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, "g.highcharts-label.highcharts-tooltip.highcharts-color-0"))
  )
  tooltip_text = tooltip.text

  # 날짜 정보 추출 (정규표현식 사용)
  date_pattern = r'<tspan style="font-size: 10px;">(.*?)</tspan>'
  date_match = re.search(date_pattern, tooltip_text)
  date = date_match.group(1) if date_match else 'No Date Found'

  # 채권 정보 추출 (정규표현식 사용)
  bond_pattern = r'([가-힣 ]+\(\d+년\)): <tspan style="font-weight: bold;">(\d+\.\d+)</tspan>'
  bond_matches = re.findall(bond_pattern, tooltip_text)

  data.append({'Date': date, 'Value': bond_matches})

  print(f"Point: ({point['x']}, {point['y']}), Tooltip: {tooltip_text}")

# 날짜순으로 정렬
data.sort(key=lambda x: x['Date'])

# 4. 메인 프레임으로 복귀
driver.switch_to.default_content()

driver.quit()

TypeError: string indices must be integers

In [ ]:
def move_to_svg_point(driver, svg_element, x, y, is_relative=False):
    if is_relative:
        current_pos = driver.execute_script("return [window.lastX || 0, window.lastY || 0];")
        x += current_pos[0]
        y += current_pos[1]

    coords = driver.execute_script("""
        var svg = arguments[0];
        var point = svg.createSVGPoint();
        point.x = arguments[1];
        point.y = arguments[2];
        var transformed = point.matrixTransform(svg.getScreenCTM());
        window.lastX = arguments[1];
        window.lastY = arguments[2];
        return [transformed.x, transformed.y];
    """, svg_element, x, y)

    action = ActionChains(driver)
    action.move_by_offset(coords[0], coords[1]).perform()

In [ ]:


# 데이터프레임 생성
df2 = pd.DataFrame(data)

display(df)
# CSV 파일로 저장
# df.to_csv('company_bond_rates.csv', index=False)

driver.quit()

[]

In [ ]:
points

[]